<h1>🚀 蒼狼 AI v2.2 - 一鍵啟動器</h1>

歡迎使用！點擊下方儲存格左側的 ▶️ **執行按鈕**來啟動您的 AI 分析平台。

**啟動流程：**
1. **環境設定**：自動下載專案程式碼並安裝所有必要的 Python 和 Node.js 套件。
2. **啟動服務**：同時啟動後端 FastAPI 和前端 Next.js 服務。
3. **建立通道**：使用 `ngrok` 建立一個安全的公開網址，以便您存取服務。
4. **嵌入介面**：**您的應用程式使用者介面將會直接顯示在此儲存格的下方**，無需離開 Colab。

---
**提示**：首次執行可能需要幾分鐘時間來下載和安裝套件，請耐心等候。

In [ ]:
#@title ▶️ 點此一鍵啟動 Wolf AI 平台
import os
import subprocess
import threading
from pyngrok import ngrok
from IPython.display import display, IFrame, HTML
import time

# --- 設定 --- 
REPO_URL = "https://github.com/hsp1234-web/wolfai_v2_jules.git"
REPO_PATH = "/content/wolfai_v2_jules"
NGROK_AUTH_TOKEN = "" # 建議填寫您的 ngrok Authtoken 以獲得更穩定連線

def stream_output(process, log_file):
    """將子進程的輸出即時寫入日誌檔案"""
    with open(log_file, 'w') as f:
        for line in iter(process.stdout.readline, ''):
            f.write(line)
            f.flush()

print("⏳ [1/5] 開始準備環境...")

if os.path.exists(REPO_PATH):
    print(f"   -> 倉庫目錄 {REPO_PATH} 已存在，正在更新...")
    subprocess.run(f'cd {REPO_PATH} && git pull', shell=True, check=True)
else:
    print(f"   -> 正在從 {REPO_URL} 下載專案...")
    subprocess.run(f'git clone {REPO_URL} {REPO_PATH}', shell=True, check=True)

print("✅ [2/5] 專案程式碼已備妥！")
print("⏳ [3/5] 正在安裝後端與前端依賴套件 (此步驟可能需要數分鐘)...")

# 安裝所有依賴
subprocess.run(f'pip install -q -r {REPO_PATH}/backend/requirements.txt', shell=True, check=True)
subprocess.run(f'npm install --prefix {REPO_PATH}/frontend', shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL) # 隱藏 npm 的大量輸出

print("✅ [4/5] 所有依賴已安裝！")
print("⏳ [5/5] 正在啟動後端與前端服務...")

# 啟動後端
backend_process = subprocess.Popen(['uvicorn', 'main:app', '--host', '0.0.0.0', '--port', '8000'], cwd=f'{REPO_PATH}/backend', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
threading.Thread(target=stream_output, args=(backend_process, 'backend.log')).start()
print("   -> 後端 FastAPI 服務已在背景啟動。")

# 啟動前端
frontend_process = subprocess.Popen(['npm', 'run', 'dev'], cwd=f'{REPO_PATH}/frontend', stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
threading.Thread(target=stream_output, args=(frontend_process, 'frontend.log')).start()
print("   -> 前端 Next.js 服務已在背景啟動。")

# 設定 ngrok
if NGROK_AUTH_TOKEN:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("   -> 正在等待服務穩定並建立 ngrok 通道...")
time.sleep(15) # 等待服務完全啟動

# 建立 ngrok 通道並顯示
try:
    public_url = ngrok.connect(3000).public_url
    print(f"
🎉🎉🎉 您的 AI 平台已上線！ 🎉🎉🎉")
    print(f"🌐 公開網址: {public_url}")
    display(HTML("<h2>↓ 您可以直接在下方操作 WolfAI 介面 ↓</h2>"))
    display(IFrame(public_url, width='100%', height=800))
except Exception as e:
    print(f"
🛑 錯誤：無法建立 ngrok 通道。請檢查您的 Authtoken 或 Colab 網路狀態。錯誤訊息: {e}")